The purpose of this notebook is to combine data from our 2 data sources.

Specifically look at the following 2 webpages, with different data on the same election:
- https://www.electionsireland.org/result.cfm?election=1977&cons=219
- https://www.irelandelection.com/election.php?elecid=11&electype=1&constitid=48

Note: The quotas are different.

For every elections we want
- Number of Constituencies
- How many consituency do we have vote data on?
- What was the quota?
- What was the votes/quota in first count?
- What was the lowest votes/quota?
- What was the highest votes/quota?
- Who transfered to who (if you have transfer data)

What parties do well in what areas in the last 25 years? 
eg total town council, dail seats, and other have been elected for a given area. eg sf:3 ff:16 fg: 14 

In [2]:
import pandas as pd
import numpy as np

### Introducing the first data set

The first dataset comes from https://www.electionsireland.org/ 

This data has date 12 columns:

- date: _str_, This is sometimes the exact date of the election, othertimes its just the year, other times its the year and the month: needs to be combined with the other data set which has the year.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election, This data needs to cleaned because it also includes intra-party elections and resignations. (Interesting data but not needed for this analysis)

- party: _str_, The party a candidate ran with, or for independents, it lists them as belonging to a party called independent.

- status: _str_, If they were elected or not. This data needs to be cleaned as it includes other information such as whether the candidate made the cutoff for expenses. 

- constituency_name: _str_, The name of the constituency the candidate ran in.

- seat: _int_, Contains the order in which the candidate was elected, if the candidate wasnt elected then its None.

- count_eliminated: _int_, Which count the candidate was either **elected** or **lost**. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- pct_of_quota_reached_with_first_pref: _float_, The % of the quota that a candidate reached with first preference votes.

- ran_unopposed: _bool_, True if the candidate ran unopposed (more common in earlier elections)

- candidate: _str_, The candidate's name

- candidate_ID: _int_, A unique ID for each candidate 


In [3]:
df1 = pd.read_parquet('electionsireland_data/ElectionsIreland_candidate.parquet')
df1 = df1.rename(columns={'ID':'candidate_ID'})
print(df1.columns)
print(df1.shape)
df1.head()

Index(['date', 'election_type', 'party', 'status', 'constituency_name', 'seat',
       'count_eliminated', 'first_pref_count', 'first_pref_pct',
       'pct_of_quota_reached_with_first_pref', 'ran_unopposed', 'candidate',
       'candidate_ID'],
      dtype='object')
(30070, 13)


,date,election_type,party,status,constituency_name,seat,count_eliminated,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,None,NaN,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,3,NaN,6299.0,0.1408,NaN,False,Seamus Pattison,1
3,1969,19th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,6041.0,0.1311,NaN,False,Seamus Pattison,1
4,1973,20th Dail,Labour,Elected,Carlow Kilkenny,4,NaN,5300.0,0.1134,NaN,False,Seamus Pattison,1


### Introducing the second dataset

The first dataset comes from https://www.irelandelection.com/ 

This data has date 10 columns:

- election: _str_, Contains a string describing the election usually just the consitituency and the date of the election

- elected: _bool_, If they were elected or not. This data needs to be combined with Status from the first dataframe. 

- party: _str_, The party a candidate ran with

- first_pref_pct: _float_, The % of all first preference votes a candidate recieved. 

- first_pref_count _int_, The count of first preference votes recieved. 

- first_pref_quota_ratio: _float_, The % of the quota that a candidate reached with first preference votes.

- year: _int_, The year of the election. *NB:* Sometimes there is two elections of the same kind in the same year, (eg 2 dail elections in 1982).

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Seanad, Dail, Bi-Election.

- candidate: _str_, The candidate's name. Unfortunatly there is 2 candidates have the same, they are essentially grouped as the same person, we fix this by linking records with the first dataset. 

- constituency: _str_, The name of the constituency the candidate ran in.

- election_type: _str_, Is the type of election the candidate was running in, eg Local, Dail, Bi-Election.

In [4]:
df2 = pd.read_parquet('irelandelection/ALL_CANDIDATES.parquet')
print(df2.columns)
print(df2.shape)

df2.head()

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'first_pref_quota_ratio', 'year', 'candidate', 'constituency',
       'election_type'],
      dtype='object')
(36243, 10)


,election,elected,party,first_pref_pct,first_pref_count,first_pref_quota_ratio,year,candidate,constituency,election_type
0,2004 Local Election - Thomastown,True,Labour Party,0.085,641,0.51,2004,Ann Phelan,Thomastown,LOCAL
1,2009 Local Election - Thomastown,True,Labour Party,0.156,1183,0.78,2009,Ann Phelan,Thomastown,LOCAL
2,2011 general election - Carlow–Kilkenny,True,Labour Party,0.109,8072,0.66,2011,Ann Phelan,Carlow–Kilkenny,GENERAL
3,2016 general election - Carlow–Kilkenny,False,Labour Party,0.063,4391,0.38,2016,Ann Phelan,Carlow–Kilkenny,GENERAL
0,1982 (Feb) general election - Carlow–Kilkenny,False,Fianna Fáil,0.017,907,0.10,1982,John McGuinness,Carlow–Kilkenny,GENERAL


### Cleaning Dataframe 1

So we need to clean the following columns in dataframe 1:
- seat and count_eliminated can be dropped as for this analysis we dont really care about the order in which candidates were elected. 
- date, this needs to be converted to a year columns.
- election_type, this should read the same as dataframe 2 with options like GENERAL, LOCAL and SEANAD
- status, will be kept, but we want a column that is a simple bool TRUE or FALSE was the candidate elected.


In [5]:
# dropping seat and count_eliminated

df1 = df1.drop(columns=['seat','count_eliminated'])
df1.head(3)

,date,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID
0,23 June 1960,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1
1,1961,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1
2,1965,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1


In [6]:
#cleaning date

def get_year_from_date_string(date_str):
    if date_str == None:
        return 0
    elif len(date_str) > 4:
        try:
            return int(date_str[-4:])
        except:
            if isinstance(date_str[-1],int):#last letter is an int
                return date_str[-4:]
    else:
        return int(date_str)

df1 = df1.reset_index().drop(columns=['index'])
df1['year'] = df1.date.apply(get_year_from_date_string)
df1 = df1.drop(columns=['date'])

df1.head(3)

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year
0,By Election,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0
1,17th Dail,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0
2,18th Dail,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0


In [7]:
#cleaning election_type

def get_election_type_from_string(election_type_str):
    if 'Town' in election_type_str or 'Local' in election_type_str :
        return 'LOCAL'
    elif 'Dail' in election_type_str:
        return 'GENERAL'
    elif 'Seanad' in election_type_str:
        return 'SEANAD'
    elif 'Westminster' in election_type_str:
        return 'Westminster'.upper()
    elif 'European' in election_type_str:
        return 'EUROPEAN'
    elif 'By Election' in election_type_str:
        return 'BI-ELECTION'
    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['election_type'] = df1.election_type.apply(get_election_type_from_string)
print(df1.election_type.unique())

['BI-ELECTION' 'GENERAL' 'EUROPEAN' None 'LOCAL' 'SEANAD' 'WESTMINSTER']


In [8]:
df1.status.unique()

array(['Not Elected', 'Elected', 'Appointed', 'Resigned', 'Disqualified',
       None, 'Co-opted', 'Candidate', 'Dublin', 'Lucan',
       'awaiting update', 'changed\xa0to',
       'Dublin University (Trinity College)', 'Died in office:',
       'Agricultural Panel', 'Gorey\xa0\xa0\xa0-\xa0\xa0\xa0Resigned',
       'Nominated by Taoiseach', 'Inishowen', '(Replaced  David McKenna)',
       '(Replaced  Michael Flynn)', 'Administrative Panel', 'Drumcliff',
       'Ballybrack\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (ill health)',
       'Kilkenny\xa0\xa0\xa0-\xa0\xa0\xa0Resigned\n            \n              (dual mandate TD)'],
      dtype=object)

In [9]:
# cleaning status
def was_elected(status):
    status = str(status) # some of the status are None 

    if status == 'Elected':
        return True
    elif status == 'Not Elected':
        return False

    else: # for the rows that represent resignations or appointments or some other event in a politicans career
        return None

df1['elected'] = df1.status.apply(was_elected)
df1.head()

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
0,BI-ELECTION,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0,False
1,GENERAL,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0,True
2,GENERAL,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0,True
3,GENERAL,Labour,Elected,Carlow Kilkenny,6041.0,0.1311,NaN,False,Seamus Pattison,1,1969.0,True
4,GENERAL,Labour,Elected,Carlow Kilkenny,5300.0,0.1134,NaN,False,Seamus Pattison,1,1973.0,True


### Cleaing dataframe 2

Renaming:
Dataframe 2 has a column called ```first_pref_quota_ratio``` which is the same information as ```pct_of_quota_reached_with_first_pref``` in dataframe 1

In [10]:
df2 = df2.rename(columns={'first_pref_quota_ratio':'pct_of_quota_reached_with_first_pref','constituency':'constituency_name'}).reset_index().drop(columns=['index'])

print(df2.columns)
print(df2.shape)

Index(['election', 'elected', 'party', 'first_pref_pct', 'first_pref_count',
       'pct_of_quota_reached_with_first_pref', 'year', 'candidate',
       'constituency_name', 'election_type'],
      dtype='object')
(36243, 10)


### Joining the two datasets:

Some of the data in Dataframe 1 is missings, hence why we are combining these 2 dataframes.
I remove entries from dataframe 1 that arent elected. 

Recall that some rows in dataframe 1 just represent a politician being appointed as minister or resigning, not always an election.

In [11]:
df1 = df1[~df1.elected.isnull()]
df1

,election_type,party,status,constituency_name,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,candidate,candidate_ID,year,elected
0,BI-ELECTION,Labour,Not Elected,Carlow Kilkenny,7678.0,0.2016,NaN,False,Seamus Pattison,1,1960.0,False
1,GENERAL,Labour,Elected,Carlow Kilkenny,4116.0,0.0954,NaN,False,Seamus Pattison,1,1961.0,True
2,GENERAL,Labour,Elected,Carlow Kilkenny,6299.0,0.1408,NaN,False,Seamus Pattison,1,1965.0,True
3,GENERAL,Labour,Elected,Carlow Kilkenny,6041.0,0.1311,NaN,False,Seamus Pattison,1,1969.0,True
4,GENERAL,Labour,Elected,Carlow Kilkenny,5300.0,0.1134,NaN,False,Seamus Pattison,1,1973.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
30065,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,5764.0,0.1284,NaN,False,William Irwin,4203,2010.0,False
30066,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,13177.0,0.2459,NaN,False,William Irwin,4203,2017.0,False
30067,WESTMINSTER,Democratic Unionist,Not Elected,Newry and Armagh,11000.0,0.2166,NaN,False,William Irwin,4203,2019.0,False
30068,BI-ELECTION,Non party/Independent,Not Elected,Cork South Central,219.0,0.0052,0.01,False,Brian McEnery,4211,1994.0,False


In [12]:
print(
    df1.shape,
    df2.shape
    )

print(sorted(set(df1.columns)))
print(sorted(set(df2.columns)))

(28590, 12) (36243, 10)
['candidate', 'candidate_ID', 'constituency_name', 'elected', 'election_type', 'first_pref_count', 'first_pref_pct', 'party', 'pct_of_quota_reached_with_first_pref', 'ran_unopposed', 'status', 'year']
['candidate', 'constituency_name', 'elected', 'election', 'election_type', 'first_pref_count', 'first_pref_pct', 'party', 'pct_of_quota_reached_with_first_pref', 'year']


In [13]:
#df1.year.unique()
df1 = df1.reindex(columns=[
    'year',
    'candidate',
    'candidate_ID',
    'constituency_name',
    'party',
    'elected',
    'election_type',
    'first_pref_count',
    'first_pref_pct',
    'pct_of_quota_reached_with_first_pref',
    'ran_unopposed',
    'status',
    ])
df1.head(3)

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
0,1960.0,Seamus Pattison,1,Carlow Kilkenny,Labour,False,BI-ELECTION,7678.0,0.2016,NaN,False,Not Elected
1,1961.0,Seamus Pattison,1,Carlow Kilkenny,Labour,True,GENERAL,4116.0,0.0954,NaN,False,Elected
2,1965.0,Seamus Pattison,1,Carlow Kilkenny,Labour,True,GENERAL,6299.0,0.1408,NaN,False,Elected


In [14]:
df2.year = df2.year.astype(float)
df2.year.unique()

array([2004., 2009., 2011., 2016., 1982., 1985., 1991., 1997., 1999.,
       2002., 2007., 2020., 2014., 2015., 1987., 1989., 1992., 1960.,
       1961., 1965., 1967., 1969., 1973., 1974., 1977., 1979., 1981.,
       1994., 1934., 1950., 1955., 1948., 1951., 1954., 1957., 1956.,
       1925., 1932., 1933., 1937., 1938., 1943., 1944., 1923., 1927.,
       1921., 1922., 1984., 2019., 1968., 1959., 1942., 1945., 1928.,
       1980., 1953., 1924., 1998., 1996., 1983., 1995., 2013., 1972.,
       2010., 1970., 1949., 1958., 1946., 1929., 1926., 1963., 1952.,
       1976., 1947., 1964., 1975., 1935., 1966., 2005., 1920., 1930.,
       2001., 2000., 1936., 1931., 1939., 1940.])

In [15]:
pd.concat([df1,df2],axis=1).sort_values('year')

ValueError: The column label 'year' is not unique.

In [16]:
combined_dataframe = pd.DataFrame()

In [39]:
import jellyfish

def are_names_the_same(name_1,name_2):
    if name_1==name_2:
        return False
    elif jellyfish.levenshtein_distance(name_1,name_2) <=2:
        return True
    else:
        return False

In [40]:
all_names = set(df1.candidate.unique()).union(set(df2.candidate.unique()))
similar_names = []

for name_1 in list(all_names):
    name_set = set(name_1)
    for name_2 in list(all_names):
        if are_names_the_same(name_1,name_2):
            name_set.add(name_2)
    similar_names.append(name_set)
        
    
    # temp_df = df1[df1.year == year]
    # temp_df2 = df2[df2.year == year]
    # for name in temp_df.candidate:
    #     mp_df2.candidate.apply(lambda name_2: jellyfish.levenstein_distance)

    # print(temp_df.head())
    # print(temp_df2.head())
    # break
    
similar_names

loop through and see if the name is the same or similar, are there records in both tables.

In [26]:
df1[df1.candidate.isin(similar_names[8])]

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
8437,1982.0,D F O'Sullivan,3433,Cork South West,Fianna Fail,False,GENERAL,2640.0,0.0804,NaN,False,Not Elected
8438,1991.0,D F O'Sullivan,3433,Skibbereen,Fianna Fail,True,LOCAL,1688.0,0.1103,0.88,False,Elected


In [38]:
df2[df2.candidate.isin(similar_names[8])]

,election,elected,party,first_pref_pct,first_pref_count,pct_of_quota_reached_with_first_pref,year,candidate,constituency_name,election_type
16905,1979 Local Election - Skibbereen,True,Fianna Fáil,0.075,1227,0.60,1979.0,D.F. O'Sullivan,Skibbereen,LOCAL
16906,1985 Local Election - Skibbereen,True,Fianna Fáil,0.131,2049,1.05,1985.0,D.F. O'Sullivan,Skibbereen,LOCAL
16907,1991 Local Election - Skibbereen,True,Fianna Fáil,0.110,1688,0.88,1991.0,D.F. O'Sullivan,Skibbereen,LOCAL


In [ ]:
temp_df.head()

,year,candidate,candidate_ID,constituency_name,party,elected,election_type,first_pref_count,first_pref_pct,pct_of_quota_reached_with_first_pref,ran_unopposed,status
18838,1880.0,T P O'Connor,10028,Galway,,True,WESTMINSTER,487.0,0.3315,NaN,False,Elected
18840,1880.0,John Lever,10029,Galway,,True,WESTMINSTER,501.0,0.3410,NaN,False,Elected
18841,1880.0,Hugh Tarpey,10030,Galway,Liberal,False,WESTMINSTER,481.0,0.3274,0.98,False,Not Elected


In [ ]:
temp_df2.head()

,election,elected,party,first_pref_pct,first_pref_count,first_pref_quota_ratio,year,candidate,constituency,election_type


Run through and say per name in 